In [7]:
from tkinter import *
from PIL import ImageTk, Image
import threading
import cv2 
import tkinter as tk


# These are all necessary in order to run.
# If the camera decides to give you crap tehn check if PIL is installed properly
# If the problem persists then check if cv2 is installed correctly. you may have to install from the terminal. 
# The command should be pip install opencv-python


In [8]:
def hz_to_ms(hz):
    """
    converts Hz to Milliseconds
    :param hz: hertz
    :type hz: int or float
    :return: milliseconds
    :rtype: int
    """
    if hz == 0:
        raise ZeroDivisionError
    return int(round(1000/hz))

In [11]:

class Controller:
    def __init__(self, root, box_1_hz, box_2_hz, stream_url = False):
        """
        Creates the graphical user interface
        :param root: the main window. Should be what you set tk.Tk() to be
        :param box_1_hz: how fast to flash box_1
        :param box_2_hz: how fast to flash box_2
        :param stream_url: what camera feed to get.
        """
        # Window stuff
        self.root = root
        self.root.title("Controller")
        root.attributes('-fullscreen', True)
        self.screen_width = self.root.winfo_screenwidth()
        self.screen_height = self.root.winfo_screenheight()
        
        # This is information about what camera you are using and the size of the video feed.
        if not stream_url:
            self.cap = cv2.VideoCapture(0)
        else:
            self.cap = cv2.VideoCapture(stream_url)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.screen_width // 3)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.screen_height // 3)
        

        # this sets up the GUI
        self.canvas = tk.Canvas(self.root, width=self.screen_width, height=self.screen_height)
        self.canvas.place(anchor='nw', x=0, y=0)
        self.video = tk.Label(self.root)
        self.video.place(anchor='center', relx=0.5, rely=0.5)
        
        self.down = self.create_flash_rect(self.screen_width // 2, self.screen_height // 2 + 450)
        self.up = self.create_flash_rect(self.screen_width // 2, self.screen_height // 2 - 450)
        # threading stuff
        # Allows for python to run tasks in parallel, with the tasks being the flashing. daemon=true basically kills the thread when you close the window.
        threading.Thread(target=self.flash, args=(self.up, hz_to_ms(box_1_hz)), daemon=True).start()
        threading.Thread(target=self.flash, args=(self.down, hz_to_ms(box_2_hz)), daemon=True).start()
        threading.Thread(target=self.open_camera, daemon=True).start()
        
    def run(self):
        """
        Runs the code
        :return: 
        """
        self.root.mainloop()

    def open_camera(self):
        """
        Black magic to run the camera.
        :return: 
        """
        # Checks if your camera is on.
        if self.cap.isOpened():
            # Capture the video frame by frame
            _, frame = self.cap.read()
            # Resize Image and convert Colors
            frame = cv2.cvtColor(cv2.resize(frame, (self.screen_width//3, self.screen_height//3)), cv2.COLOR_BGR2RGBA)
            # Capture the latest frame and transform to image
            self.photo_image = ImageTk.PhotoImage(image=Image.fromarray(frame))
            # Displaying camera feed onto the gui
            self.video.configure(image=self.photo_image)
        # Waits 10 ms before running this code again
        self.root.after(10, self.open_camera)
    
    def create_flash_rect(self, x, y, size=65):
        """
        Creates a rectangle with specified size at a location
        :param x: x location
        :param y: y location
        :param size: the actual size is size * 2
        :return: a black rectangle centered at x,y with width=2*size and height=*size
        """
        # Create a flashing rectangle centered at (x, y).
        rect = self.canvas.create_rectangle(
            x - size, y - size, x + size, y + size, fill='black'
        )
        return rect

    def flash(self, rect, interval:int):
        """
        Makes an item flash at a specified interval. Straight Voodoo Magic.
        :param rect: the item you want to flash
        :type rect: tuple
        :param interval: in milliseconds, how often to flash the item.
        :type interval: int
        :return:
        """
        # Toggle the fill color of a rectangle at a specified interval
        current_fill = self.canvas.itemcget(rect, 'fill')
        self.canvas.itemconfig(rect, fill='white' if current_fill == 'black' else 'black')
        self.root.after(interval, lambda: self.flash(rect, interval))
        
# Before running Connect to the Car. 
# Initializes the Code


window = tk.Tk()
RC_Controller = Controller(window, 12, 15, stream_url='http://192.168.4.1/Test')
RC_Controller.run()

"""
___Notes___
- If the GUI gives you crap about running, then try to check if the stream_url is working.
- If the GUI gives you crap about stopping, try turning your camera off and on again.
"""

KeyboardInterrupt: 